# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from scraper import fetch_website_links, fetch_website_contents
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-5-nano'
openai = OpenAI()

API key looks good so far


In [4]:
links = fetch_website_links("https://edwarddonner.com")
links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 '

## First step: Have GPT-5-nano figure out which links are relevant

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [2]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [5]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [6]:
print(get_links_user_prompt("https://edwarddonner.com"))


Here is the list of links on the website https://edwarddonner.com -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/
https://edwar

In [7]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [8]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'company homepage', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'partner / company page',
   'url': 'https://nebula.io/?utm_source=ed&utm_medium=referral'},
  {'type': 'linkedin', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'twitter', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook', 'url': 'https://www.facebook.com/edward.donner.52'},
  {'type': 'patent',
   'url': 'https://patents.google.com/patent/US20210049536A1/'}]}

In [12]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [9]:
select_relevant_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/connect-four/'},
  {'type': 'project page', 'url': 'https://edwarddonner.com/outsmart/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'patent',
   'url': 'https://patents.google.com/patent/US20210049536A1/'},
  {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/in/eddonner/'},
  {'type': 'article',
   'url': 'https://edwarddonner.com/2025/09/15/ai-in-production-gen-ai-and-agentic-ai-on-aws-at-scale/'},
  {'type': 'article',
   'url': 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/'},
  {'type': 'article',
   'url': 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/'},
  {'type': 'article',
   'url': 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineer

In [10]:
select_relevant_links("https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 3 relevant links


{'links': [{'type': 'brand page', 'url': 'https://huggingface.co/brand'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT-5-nano

In [ ]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)
    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"
    for link in relevant_links['links']:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link["url"])
    return result

In [14]:
print(fetch_page_and_all_relevant_links("https://huggingface.co"))

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links
## Landing Page:

Hugging Face – The AI community building the future.

Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Try HuggingChat Omni – Chat with AI 💬
Get started with Inference in seconds 🚀
Reachy Mini: The Open Robot for AI Builders
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
PaddlePaddle/PaddleOCR-VL
Updated
about 9 hours ago
•
3.82k
•
733
deepseek-ai/DeepSeek-OCR
Updated
about 18 hours ago
•
111
•
476
nanonets/Nanonets-OCR2-3B
Updated
5 days ago
•
12.8k
•
341
Qwen/Qwen3-VL-8B-Instruct
Updated
5 days ago
•
74.5k
•
227
inclusionAI/Ring-1T
Updated
about 10 hours ago
•
435
•
184
Browse 1M+ models
Spaces
Running
417
417
veo3.1-fast
🐨
Generate videos from text or images
Running
388
388

In [23]:
# brochure_system_prompt = """
# You are an assistant that analyzes the contents of several relevant pages from a company website
# and creates a short brochure about the company for prospective customers, investors and recruits.
# Respond in markdown without code blocks.
# Include details of company culture, customers and careers/jobs if you have the information.
# """

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short, humorous, entertaining, witty brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"""
You are looking at a company called: {company_name}
Here are the contents of its landing page and other relevant pages;
use this information to build a short brochure of the company in markdown without code blocks.\n\n
"""
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


'\nYou are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages;\nuse this information to build a short brochure of the company in markdown without code blocks.\n\n\n## Landing Page:\n\nHugging Face – The AI community building the future.\n\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nTry HuggingChat Omni – Chat with AI 💬\nGet started with Inference in seconds 🚀\nReachy Mini: The Open Robot for AI Builders\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nPaddlePaddle/PaddleOCR-VL\nUpdated\nabout 9 hours ago\n•\n3.82k\n•\n733\ndeepseek-ai/DeepSeek-OCR\nUpdated\nabout 18 hours ago\n•\n111\n•\n476\nnanonets/Nanonets-OCR2-3B\nUpdated\n5 days ago\n•\n12.8k\n•\n341\nQwen/Qwen3-VL-8B-Instruct\nUpdated\n5 days ago\n•\n74

In [19]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [20]:
create_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 13 relevant links


# Hugging Face Brochure

---

## About Hugging Face

Hugging Face is a vibrant AI and machine learning community platform dedicated to building the future of artificial intelligence through open collaboration. As a central hub for sharing, discovering, and experimenting with machine learning models, datasets, and applications, Hugging Face empowers machine learning engineers, scientists, and enthusiasts worldwide to learn, innovate, and co-create an open and ethical AI future.

### Mission
To democratize good machine learning, one commit at a time.

---

## What We Offer

### Hub for Machine Learning Collaboration
- **Models**: Access and contribute to over 1 million open-source ML models covering text, image, video, audio, and even 3D modalities.
- **Datasets**: Browse and share from a collection of 250,000+ datasets for various AI tasks.
- **Spaces**: Build and deploy 400,000+ AI applications collaboratively, from text-to-video generation to many other innovative use cases.
- **Community**: Join a fast-growing, passionate community driving the AI revolution through open sharing and collaboration.

### Tools & Solutions
- **Open Source Stack**: Utilize powerful open-source tools to accelerate your AI development from research to deployment.
- **Enterprise Solutions**: Benefit from tailored compute and enterprise-grade offerings to meet your team or organization's AI needs.
- **HuggingChat Omni**: Try conversational AI with HuggingChat—a new way to chat and interact with AI models.

---

## Company Culture

- **Collaborative & Inclusive**: Hugging Face fosters an open environment where knowledge sharing and community input drive innovation.
- **Open & Ethical AI**: Dedicated to building AI technologies transparently and responsibly.
- **Learning & Growth**: Emphasizes continuous learning with initiatives like the "Hugging Face Fundamentals" learning track on DataCamp.
- **Global Impact**: Welcomes contributions from a diverse and worldwide community of over 64,000 followers and tens of thousands of active contributors.

---

## Careers & Join Us

Are you passionate about machine learning and open source? Hugging Face is growing rapidly with a team of nearly 200 talented scientists, engineers, and community managers exploring the cutting edge of AI.

- **Current Openings**: Various roles across research, engineering, product, and community engagement.
- **Why Work Here?**
  - Be part of the AI revolution shaping the future.
  - Collaborate with a dedicated and inspiring team.
  - Engage in meaningful work that values ethics and accessibility.
  - Enjoy a flexible, supportive work environment.

Interested in joining? Visit [huggingface.co/jobs](https://huggingface.co/jobs) or contact their team for opportunities.

---

## Who Uses Hugging Face?

Hugging Face serves:

- Machine Learning Engineers and Data Scientists
- AI Researchers and Academics
- Enterprises seeking scalable AI solutions
- Developers building AI-powered applications
- Enthusiasts and learners looking to build AI skills and portfolios

Trusted by thousands of organizations and individual contributors, Hugging Face is the go-to platform for accelerating ML innovation.

---

## Key Stats

- 1M+ Machine Learning models hosted
- 250k+ datasets available
- 400k+ AI applications deployed
- 64,000+ community followers and counting
- 198 passionate team members driving the tech forward

---

## Contact & Social

Website: [huggingface.co](https://huggingface.co)  
Twitter: [@huggingface](https://twitter.com/huggingface)  
GitHub: [github.com/huggingface](https://github.com/huggingface)  
LinkedIn: [Hugging Face](https://linkedin.com/company/huggingface)  
Discord: Active community discussions and support  

For press inquiries or business contacts, reach out via the website.

---

**Hugging Face** — The AI community building the future, together.  
Join us to explore, create, and innovate in the open AI ecosystem.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [21]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [22]:
stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 10 relevant links


# Hugging Face: Building the Future of AI Together

---

## Who We Are

Hugging Face is the leading collaboration platform at the heart of the AI revolution, empowering machine learning engineers, scientists, and enthusiasts worldwide. Our community-driven hub is the central place to **share, explore, discover, and experiment** with open-source machine learning models, datasets, and applications. We strive to build an open, ethical AI future by fostering innovation and collaboration across all AI modalities—text, image, video, audio, and even 3D.

---

## Our Platform & Offerings

- **Models:** Access and contribute to over **1 million open-source ML models**, updated and enhanced regularly by our vibrant community.
- **Datasets:** Explore a vast repository of **250,000+ datasets** spanning multiple domains, facilitating training and research.
- **Spaces:** Share and run your ML applications with over **400,000 apps** hosted for public use.
- **Community:** Join a fast-growing, active community that connects AI builders through discussion forums, events, and collaborative projects.
- **HuggingChat Omni:** Our AI-powered chat product that offers intuitive conversational AI experiences.
- **Inference in Seconds:** Deploy machine learning models quickly with our scalable inference APIs and infrastructure.
- **Enterprise Solutions:** Advanced, secure AI platforms tailored for teams and organizations with features like Single Sign-On, resource and token management, granular access control, audit logs, dedicated support, and private storage.

---

## Why Choose Hugging Face?

- **Collaborative Culture:** At Hugging Face, we believe in open science and co-creation. Our culture promotes transparency, inclusion, and learning. We empower a global network of developers, researchers, and AI practitioners to build the future of machine learning together.
- **Innovative Tools:** Powered by a talented science team pushing the edge of AI technology, we create some of the world’s most popular open-source ML libraries and tools.
- **Diversity of AI Modalities:** Whether your work involves natural language processing, computer vision, audio analysis, or 3D modeling, our platform supports every dimension of AI research and deployment.
- **Support for Developers and Enterprises:** From individual ML engineers building portfolios to large enterprises needing robust, secure AI infrastructure—Hugging Face scales with you.

---

## Our Customers and Community

We serve a diverse user base from hobbyists and independent researchers to major corporations and academic institutions. Our Enterprise Hub caters to teams requiring:

- Enterprise-grade security and compliance
- Secure Single Sign-On and identity integration
- Granular repository and data access control
- Customizable compute scalability
- Dedicated support and analytics dashboards

---

## Careers at Hugging Face

Join us as we shape AI’s future. We seek passionate individuals across roles in:

- Machine Learning Engineering & Research
- Software Development
- DevOps and Infrastructure
- Community Engagement and Developer Relations
- Product Management and Design

At Hugging Face, you'll work alongside industry experts in a supportive, open culture dedicated to innovation and ethical AI development.

Explore open positions and become part of our mission to democratize AI: [Hugging Face Careers](https://huggingface.co/jobs)

---

## Get Started

Discover the power of collaborative AI:

- Browse models, datasets, and applications at [huggingface.co](https://huggingface.co)
- Sign Up to build your portfolio and start contributing to the world’s largest machine learning community
- Unlock enterprise-grade features tailored for your organization

---

## Contact & Social

- GitHub: github.com/huggingface
- Twitter: twitter.com/huggingface
- LinkedIn: linkedin.com/company/huggingface
- Discord community for live discussions and support

---

Together, let’s build the future of AI—open, collaborative, and ethical.

**Hugging Face**  
*The AI community building the future.*

In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Selecting relevant links for https://huggingface.co by calling gpt-5-nano
Found 16 relevant links


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# Welcome to Hugging Face: Where AI Gets Its Hug!

## Who Are We?  
At Hugging Face, we're not just a company — we're a buzzing AI playground for the machine learning community. Think of us as the friendly neighborhood hangout where ML engineers, data scientists, and AI enthusiasts come to share, collaborate, and build the future of AI *together*. We’re the ultimate *hug* for your AI ambitions.

## What We Do  
- **1 Million+ Models:** From text and images to videos and 3D, our library is a treasure trove for AI aficionados. Want to create a video from a single image? There’s a model for that. Need to build the next ChatGPT? We’ve got you covered.
- **250,000+ Datasets:** Dive into datasets that keep your algorithms sharp and your insights sharper.
- **400,000+ Applications (Spaces):** Browse creative and cutting-edge AI-powered apps that are always running, always evolving.
- **Open-Source Stack:** Move faster with the magic of open source. Our platform is all about empowering creators with tools that flex, scale, and perform.
- **Enterprise Solutions:** Got a team that needs serious AI horsepower? Our paid Compute and Enterprise services will turbocharge your projects.

## Meet Reachy Mini  
Ever wanted your very own open robot sidekick? Meet Reachy Mini — your AI buddy for building robots that do the things you dream about. Because the future isn’t just smart, it’s friendly and ready to help.

## Company Culture: The Purr-fect Ecosystem  
- **Community-Driven:** We’re powered by a vibrant, fast-growing AI community who love to experiment, share, and innovate.  
- **Open & Ethical:** Building AI isn’t just about cool models; it’s about doing it *right*. Hugging Face champions ethical AI development with open source at its heart.  
- **Fun & Friendly:** We believe work should feel a bit like play. Whether it’s coding marathons or brainstorming sessions, expect good vibes and great ideas.  
- **Build Your Portfolio:** Share your creations, get feedback, and build your ML profile — all within the same welcoming ecosystem.

## For Our Future AI Legends: Careers at Hugging Face  
- Join a team that’s redefining AI collaboration globally.
- Work on projects that actually *matter* — open-source innovation, tooling, and community building.
- Enjoy a culture that values curiosity, creativity, and teamwork.
- Get access to cutting-edge tech, supportive mentorship, and a truly global, passionate community.

## Why Hugging Face?  
Because here, AI isn't just artificial intelligence — it’s *Amazing Innovation* in a big, warm community hug.

---

**Ready to explore the future?**  
Jump in. Browse models. Share your dataset. Build your app. Or just chat with our AI over at HuggingChat Omni.  

Visit [huggingface.co](https://huggingface.co) and get your AI groove on today!

---

*Hugging Face*  
*The AI community building the future — one hug, one model, and one dataset at a time.*  
**#MachineLearning #AI #Community #OpenSource #EthicalAI**

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>